In [8]:
import cv2
import os
from model import get_angles_from_model
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import get_angles   

pose_model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
key_frames_angles = []

In [9]:
excersise_name = "Bicep_Curls"

In [10]:
if not os.path.exists(excersise_name):
    os.makedirs(excersise_name)
    print(f"Directory '{excersise_name}' created.")

In [11]:
threshold_angle = 45

In [12]:
def get_angles_from_model(model,image_path):
   image = tf.io.read_file(image_path)
   image = tf.image.decode_jpeg(image)
   image = tf.expand_dims(image, axis=0)
   image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

   movenet = model.signatures['serving_default']

   outputs = movenet(image)

   keypoints = outputs['output_0'].numpy().reshape(-1,3)

   angles = get_angles.get_angles(keypoints)
   return angles

def process_frame(frame, count, angles):
    frame_resized = cv2.resize(frame, (256, 256))
    filename = f"pose{count}.jpg"
    filename = os.path.join(excersise_name,filename)
    cv2.imwrite(filename, frame_resized)
    angles1 = get_angles_from_model(pose_model, filename)  # Pass the filename to the model
    angle_diff = sum(abs(angles[key] - angles1[key]) for key in angles)
    if angle_diff < threshold_angle:
        os.remove(filename)
    else:
        angles = angles1
        key_frames_angles.append(angles)
    return angles

In [13]:
def main():
    video = cv2.VideoCapture("video.mp4")
    count = 0
    ret, frame = video.read()
    
    if not ret:
        print("Failed to read video")
        return
    
    initial_filename = "initial_pose.jpg"
    cv2.imwrite(initial_filename, cv2.resize(frame, (256, 256)))
    angles = get_angles_from_model(pose_model,initial_filename)
    count += 1
    
    while True:
        ret, frame = video.read()
        if not ret:
            break
        
        angles = process_frame(frame, count, angles)
        
        count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    video.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [14]:
import csv
csv_file = "key_frame_angles.csv"
csv_file = os.path.join(excersise_name,csv_file)
# Extract the headers from the first dictionary (keys of the dictionary)
headers = key_frames_angles[0].keys()

# Write data to CSV
with open(csv_file, "w", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    
    # Write the header
    writer.writeheader()
    
    # Write the data
    writer.writerows(key_frames_angles)